ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'


In [2]:
# cell 1
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [3]:
# cell 2
CFG = {
    'IMG_SIZE': 224,
    'BATCH_SIZE': 64,
    'EPOCHS': 100,
    'LEARNING_RATE': 1e-4,
    'PATIENCE': 10,
    'SEED': 42
}

In [4]:
# cell 3
torch.manual_seed(CFG['SEED'])
np.random.seed(CFG['SEED'])
random.seed(CFG['SEED'])
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# cell 4
class CustomImageDataset(Dataset):
    def __init__(self, file_list, label_list, transform):
        self.file_list = file_list
        self.label_list = label_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx]).convert('RGB')
        image = self.transform(image)
        label = self.label_list[idx]
        return image, label

In [6]:
# cell 5
transform_rgb = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

transform_gray = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

In [7]:
# cell 6
from glob import glob

class_dirs = sorted(os.listdir('./train'))
file_paths, labels = [], []
for i, cls in enumerate(class_dirs):
    files = glob(f'./train/{cls}/*.jpg')
    file_paths.extend(files)
    labels.extend([i] * len(files))

train_x, val_x, train_y, val_y = train_test_split(file_paths, labels, test_size=0.2, stratify=labels, random_state=CFG['SEED'])

train_dataset_rgb = CustomImageDataset(train_x, train_y, transform_rgb)
val_dataset_rgb = CustomImageDataset(val_x, val_y, transform_rgb)
train_dataset_gray = CustomImageDataset(train_x, train_y, transform_gray)
val_dataset_gray = CustomImageDataset(val_x, val_y, transform_gray)

train_loader_rgb = DataLoader(train_dataset_rgb, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader_rgb = DataLoader(val_dataset_rgb, batch_size=CFG['BATCH_SIZE'], shuffle=False)
train_loader_gray = DataLoader(train_dataset_gray, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader_gray = DataLoader(val_dataset_gray, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [10]:
import torchvision.models as models
import timm
import torch.nn as nn
import torch
import torch.optim as optim

# 클래스 수 자동 계산
num_classes = len(class_dirs)

class EnsembleModel(nn.Module):
    def __init__(self, num_classes):
        super(EnsembleModel, self).__init__()

        self.model1 = models.densenet201(pretrained=True)
        self.model1.classifier = nn.Identity()
        self.dim1 = 1920

        self.model2 = models.vgg16(pretrained=True)
        self.model2.classifier = nn.Sequential(*list(self.model2.classifier.children())[:-1])
        self.dim2 = 4096

        self.model3 = timm.create_model('xception', pretrained=True, num_classes=0)
        self.dim3 = 2048

        self.classifier = nn.Linear(self.dim1 + self.dim2 + self.dim3, num_classes)

    def forward(self, x):
        x1 = self.model1(x)
        x2 = self.model2(x)
        x3 = self.model3(x)
        x_cat = torch.cat((x1, x2, x3), dim=1)
        out = self.classifier(x_cat)
        return out

# 모델 초기화 (클래스 수 인자로 전달)
model_rgb = EnsembleModel(num_classes).to(device)
model_gray = EnsembleModel(num_classes).to(device)

# 옵티마이저와 손실 함수 정의
optimizer_rgb = optim.Adam(model_rgb.parameters(), lr=CFG['LEARNING_RATE'])
optimizer_gray = optim.Adam(model_gray.parameters(), lr=CFG['LEARNING_RATE'])

criterion = nn.CrossEntropyLoss()

c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. Yo

In [18]:
# cell 8
best_val_loss_rgb = float('inf')
best_val_loss_gray = float('inf')
patience_counter = 0

for epoch in range(CFG['EPOCHS']):
    model_rgb.train()
    model_gray.train()
    train_loss_rgb, train_loss_gray = 0, 0

    for (x_rgb, y), (x_gray, _) in zip(train_loader_rgb, train_loader_gray):
        x_rgb, y = x_rgb.to(device), y.to(device)
        x_gray = x_gray.to(device)

        optimizer_rgb.zero_grad()
        output_rgb = model_rgb(x_rgb)
        loss_rgb = criterion(output_rgb, y)
        loss_rgb.backward()
        optimizer_rgb.step()
        train_loss_rgb += loss_rgb.item()

        optimizer_gray.zero_grad()
        output_gray = model_gray(x_gray)
        loss_gray = criterion(output_gray, y)
        loss_gray.backward()
        optimizer_gray.step()
        train_loss_gray += loss_gray.item()

    model_rgb.eval()
    model_gray.eval()
    val_loss_rgb, val_loss_gray = 0, 0

    with torch.no_grad():
        for (x_rgb, y), (x_gray, _) in zip(val_loader_rgb, val_loader_gray):
            x_rgb, y = x_rgb.to(device), y.to(device)
            x_gray = x_gray.to(device)

            output_rgb = model_rgb(x_rgb)
            output_gray = model_gray(x_gray)
            val_loss_rgb += criterion(output_rgb, y).item()
            val_loss_gray += criterion(output_gray, y).item()

    print(f"Epoch {epoch+1}: RGB Loss={val_loss_rgb:.4f} / GRAY Loss={val_loss_gray:.4f}")

    if val_loss_rgb < best_val_loss_rgb:
        best_val_loss_rgb = val_loss_rgb
        torch.save(model_rgb.state_dict(), 'best_rgb_model.pth')

    if val_loss_gray < best_val_loss_gray:
        best_val_loss_gray = val_loss_gray
        torch.save(model_gray.state_dict(), 'best_gray_model.pth')

    if val_loss_rgb >= best_val_loss_rgb and val_loss_gray >= best_val_loss_gray:
        patience_counter += 1
        if patience_counter >= CFG['PATIENCE']:
            print("Early stopping triggered")
            break
    else:
        patience_counter = 0

RuntimeError: Numpy is not available